In [ ]:
import sys
sys.path.append("../")

import src.paths as PATHS
import src.constants as CONST

import geopandas as gpd
import folium

# Relevant floodplains

Areas chosen to study the river bank erosion.

In [ ]:
areas = gpd.read_file(PATHS.DATA_DIR / "VO155184_Scope_Pilot_Bankerosion_20241129/VO155184_Scope_Pilot_Bankerosion_20241129.shp")
areas.info()

In [ ]:
display(areas)

In [ ]:
for _, row in areas.iterrows():
    print(f"{row['naam'] + ':':<27} {row['geometry'].area / 1e6:>6.2f}km\N{SUPERSCRIPT TWO}")

In [ ]:
m = folium.Map(location=[CONST.CENTRE_NL_LAT, CONST.CENTRE_NL_LON], zoom_start=CONST.DEFAULT_NL_ZOOM)

folium.GeoJson(areas.to_crs(epsg=CONST.EPSG_WGS84)).add_to(m)

m